In [1]:
import sys, rospy, tf, moveit_commander, random
from geometry_msgs.msg import Pose, Point, Quaternion 
from math import pi
import time
from sensor_msgs.msg import LaserScan
from geometry_msgs.msg import Twist
import smach
import ros_numpy
from utils_evasion import *

In [2]:
#This is optional 
#import cv2 
import tf as tf
import tf2_ros as tf2
import rospy
import numpy as np
import ros_numpy
from std_msgs.msg import String
#from tmc_msgs.msg import Voice
#from geometry_msgs.msg import Twist, WrenchStamped, TransformStamped
#from sensor_msgs.msg import Image as ImageMsg, PointCloud2
import tmc_control_msgs.msg
import trajectory_msgs.msg
class TF_MANAGER():
    def __init__(self):
        self._tfbuff = tf2.Buffer()
        self._lis = tf2.TransformListener(self._tfbuff)
        self._tf_static_broad = tf2.StaticTransformBroadcaster()
        self._broad = tf2.TransformBroadcaster()

    def _fillMsg(self, pos = [0,0,0], rot = [0,0,0,1] ,point_name ='', ref="map"):
        TS = TransformStamped()
        TS.header.stamp = rospy.Time.now()
        TS.header.frame_id = ref
        TS.child_frame_id = point_name
        TS.transform.translation.x = pos[0]
        TS.transform.translation.y = pos[1]
        TS.transform.translation.z = pos[2]
        TS.transform.rotation.x = rot[0]
        TS.transform.rotation.y = rot[1]
        TS.transform.rotation.z = rot[2]
        TS.transform.rotation.w = rot[3]
        return TS

    def pub_tf(self, pos = [0,0,0], rot = [0,0,0,1] ,point_name ='', ref="map"):
        dinamic_ts = self._fillMsg(pos, rot, point_name, ref)
        self._broad.sendTransform(dinamic_ts)

    def pub_static_tf(self, pos = [0,0,0], rot = [0,0,0,1] ,point_name ='', ref="map"):
        static_ts = self._fillMsg(pos, rot, point_name, ref)
        self._tf_static_broad.sendTransform(static_ts)

    def change_ref_frame_tf(self, point_name = '', new_frame = 'map'):
        try:
            traf = self._tfbuff.lookup_transform(new_frame, point_name, rospy.Time(0))
            translation, rotational = self.tf2_obj_2_arr(traf)
            self.pub_static_tf(pos = translation, rot = rotational, point_name = point_name, ref = new_frame)
            return True
        except:
            return False

    def getTF(self, target_frame='', ref_frame='map'):
        try:
            tf = self._tfbuff.lookup_transform(ref_frame, target_frame, rospy.Time(0))
            return self.tf2_obj_2_arr(tf)
        except:
            return [False,False]

    def tf2_obj_2_arr(self, transf):
        pos = []
        pos.append(transf.transform.translation.x)
        pos.append(transf.transform.translation.y)
        pos.append(transf.transform.translation.z)
    
        rot = []
        rot.append(transf.transform.rotation.x)
        rot.append(transf.transform.rotation.y)
        rot.append(transf.transform.rotation.z)
        rot.append(transf.transform.rotation.w)

        return [pos, rot]
class GRIPPER():
    def __init__(self):
        self._grip_cmd_pub = rospy.Publisher('/hsrb/gripper_controller/command',
                               trajectory_msgs.msg.JointTrajectory, queue_size=100)
        self._grip_cmd_force = rospy.Publisher('/hsrb/gripper_controller/grasp/goal',
        			tmc_control_msgs.msg.GripperApplyEffortActionGoal, queue_size=100)
        			
        self._joint_name = "hand_motor_joint"
        self._position = 0.5
        self._velocity = 0.5
        self._effort = 0.0
        self._duration = 1

    def _manipulate_gripper(self):
        traj = trajectory_msgs.msg.JointTrajectory()
        traj.joint_names = [self._joint_name]
        p = trajectory_msgs.msg.JointTrajectoryPoint()
        p.positions = [self._position]
        p.velocities = [self._velocity]
        p.accelerations = []
        p.effort = [self._effort]
        p.time_from_start = rospy.Duration(self._duration)
        traj.points = [p]
        self._grip_cmd_pub.publish(traj)
        
    def _apply_force(self):
        app_force = tmc_control_msgs.msg.GripperApplyEffortActionGoal()
        app_force.goal.effort = -0.5
        self._grip_cmd_force.publish(app_force)
        
    def change_velocity(self, newVel):
        self._velocity = newVel
    
    def open(self):
        self._position = 1.23
        self._effort = 0
        self._manipulate_gripper()

    def steady(self):
        self._position = -0.82
        self._effort = -0.3
        self._manipulate_gripper()
        
    def close(self):
        self._position = -0.82
        self._effort = -0.3
        self._manipulate_gripper()
        self._apply_force()
        rospy.sleep(0.8)

In [3]:
orient = [Quaternion (*tf.transformations.quaternion_from_euler(pi, -pi/2, -pi/2)), Quaternion (*tf.transformations.quaternion_from_euler(pi, -pi/2, -pi/2))]
pose = [Pose(Point( 0.5, -0.5, 1.3), orient[0]), Pose(Point( 0.5, -0.5, 1.3), orient[0])]



In [4]:
moveit_commander.roscpp_initialize(sys.argv)
#rospy.init_node('r2_wave_arm', anonymous=True)
rospy.init_node("takeshi_smach")
base_vel_pub = rospy.Publisher('/hsrb/command_velocity', Twist, queue_size=10)
laser = Laser() 

In [5]:
arm =  moveit_commander.MoveGroupCommander('arm')
#arm.set_named_target('go')
#arm.go()

[ WARN] [1668660655.084567209, 2356.016000000]: Link hand_l_finger_vacuum_frame has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1668660655.090725598, 2356.016000000]: Link head_l_stereo_camera_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1668660655.090798007, 2356.016000000]: Link head_r_stereo_camera_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1668660655.093747478, 2356.016000000]: Group state 'neutral' doesn't specify all group joints in group 'arm'. wrist_ft_sensor_frame_joint is missing.
[ WARN] [1668660655.093779519, 2356.016000000]: Group state 'go' doesn't specify all group joints in group 'arm'. wrist_ft_sensor_frame_joint is missing.

In [6]:
# We get the joint values from the group and change some of the values:
#Forward kinematics
#establecemos nuestro 0,0
joint_goal = arm.get_current_joint_values()
joint_goal[0] = 0.0
joint_goal[1] = 0.0
joint_goal[2] = 0.0
joint_goal[3] = 0.0
joint_goal[4] = 0.0
joint_goal[5] = 0.0

arm.set_joint_value_target(joint_goal)
arm.go()

[ WARN] [1668660664.569085408, 2358.448000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2358.448000 according to authority unknown_publisher
[ WARN] [1668660665.708858820, 2358.841000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2358.841000 according to authority unknown_publisher
[ WARN] [1668660668.738395198, 2359.537000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2359.537000 according to authority unknown_publisher
[ WARN] [1668660671.374355333, 2360.372000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2360.372000 according to authority unknown_publisher
[ WARN] [1668660681.108071045, 2363.142000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2363.166000 according to authority unknown_publisher


True

[ WARN] [1668660695.842522896, 2367.208000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2367.208000 according to authority unknown_publisher


In [7]:
#para usar el gripper
grip = GRIPPER()

[ WARN] [1668660697.647657564, 2367.729000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2367.763000 according to authority unknown_publisher
[ WARN] [1668660699.724082949, 2368.565000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2368.565000 according to authority unknown_publisher


In [8]:
#abrimos gripper
grip.open()

In [9]:
#sube cuerpo y baja el brazo a posicion deseada
joint_goal = arm.get_current_joint_values()
joint_goal[0] = 0.15
joint_goal[1] = -2.1

arm.set_joint_value_target(joint_goal)
arm.go()

[ WARN] [1668660706.333130202, 2370.364000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2370.429000 according to authority unknown_publisher
[ WARN] [1668660711.658861394, 2371.726000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2371.732000 according to authority unknown_publisher
[ WARN] [1668660713.998824834, 2372.432000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2372.479000 according to authority unknown_publisher
[ WARN] [1668660723.086660473, 2375.032000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2375.032000 according to authority unknown_publisher
[ WARN] [1668660728.634645342, 2376.607000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2376.607000 according to authority unknown_publisher
[ WARN] [1668660728.785005790, 2376.646000000]: TF_REPEATED_DATA ignoring data with redundant timest

True

In [10]:
#gira muñeca hacia abajo para tomar la piedra
joint_goal = arm.get_current_joint_values()
joint_goal[3] = -1.0

arm.set_joint_value_target(joint_goal)
arm.go()

[ WARN] [1668660786.460374174, 2392.726000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2392.726000 according to authority unknown_publisher
[ WARN] [1668660786.671115261, 2392.814000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2392.814000 according to authority unknown_publisher
[ WARN] [1668660794.036750090, 2394.860000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2394.860000 according to authority unknown_publisher
[ WARN] [1668660804.782554420, 2397.504000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2397.504000 according to authority unknown_publisher
[ WARN] [1668660806.734024785, 2397.866000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2397.895000 according to authority unknown_publisher
[ WARN] [1668660808.170417031, 2398.432000000]: TF_REPEATED_DATA ignoring data with redundant timest

True

In [11]:
#baja a tomar la piedra
joint_goal = arm.get_current_joint_values()
joint_goal[0] = 0.09

arm.set_joint_value_target(joint_goal)
arm.go()

[ WARN] [1668660818.212428296, 2401.249000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2401.249000 according to authority unknown_publisher
[ WARN] [1668660822.950724915, 2402.786000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2402.786000 according to authority unknown_publisher
[ WARN] [1668660823.985872173, 2403.121000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2403.138000 according to authority unknown_publisher
[ WARN] [1668660826.728373411, 2403.678000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2403.678000 according to authority unknown_publisher


True

[ WARN] [1668660848.630474066, 2408.897000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2408.936000 according to authority unknown_publisher
[ WARN] [1668660851.940006997, 2409.929000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2409.968000 according to authority unknown_publisher
[ WARN] [1668660881.806635556, 2418.181000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2418.181000 according to authority unknown_publisher
[ WARN] [1668660907.319190352, 2425.152000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2425.178000 according to authority unknown_publisher
[ WARN] [1668660916.539630606, 2427.356000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2427.382000 according to authority unknown_publisher
[ WARN] [1668660928.320624804, 2430.828000000]: TF_REPEATED_DATA ignoring data with redundant timest

In [12]:
#cierra el gripper tomando la piedra
grip.close()

In [13]:
#nos regresa a 0,0 con la piedra
joint_goal = arm.get_current_joint_values()
joint_goal[0] = 0.0
joint_goal[1] = 0.0
joint_goal[2] = 0.0
joint_goal[3] = 0.0
joint_goal[4] = 0.0
joint_goal[5] = 0.0

arm.set_joint_value_target(joint_goal)
arm.go()

[ WARN] [1668661136.151873709, 2480.284000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2480.284000 according to authority unknown_publisher
[ WARN] [1668661270.131760771, 2504.632000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2504.641000 according to authority unknown_publisher


True

[ WARN] [1668661338.544976420, 2518.731000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2518.791000 according to authority unknown_publisher
[ WARN] [1668661339.252867234, 2518.930000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2518.930000 according to authority unknown_publisher
[ WARN] [1668661340.242940793, 2519.228000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2519.228000 according to authority unknown_publisher
[ WARN] [1668661357.180045356, 2522.793000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2522.793000 according to authority unknown_publisher
[ WARN] [1668661382.304806551, 2527.965000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2527.985000 according to authority unknown_publisher
[ WARN] [1668661395.944604072, 2531.017000000]: TF_REPEATED_DATA ignoring data with redundant timest

In [14]:
####caminamos ala mesa de las piedras#####3

In [15]:
#####tirar piedra######

In [16]:
joint_goal = arm.get_current_joint_values()
joint_goal[0] = 0.45

arm.set_joint_value_target(joint_goal)
arm.go()

[ WARN] [1668661567.077538800, 2567.268000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2567.301000 according to authority unknown_publisher
[ WARN] [1668661573.531042610, 2568.548000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2568.548000 according to authority unknown_publisher
[ WARN] [1668661605.481974457, 2574.753000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2574.753000 according to authority unknown_publisher
[ WARN] [1668661609.778373815, 2575.985000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2576.007000 according to authority unknown_publisher
[ WARN] [1668661621.955797070, 2578.581000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2578.581000 according to authority unknown_publisher


True

[ WARN] [1668661635.154715455, 2581.629000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2581.690000 according to authority unknown_publisher


In [17]:
joint_goal = arm.get_current_joint_values()
joint_goal[1] = -2.1

arm.set_joint_value_target(joint_goal)
arm.go()

[ WARN] [1668661662.743431283, 2588.272000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2588.292000 according to authority unknown_publisher
[ WARN] [1668661712.678019555, 2598.378000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2598.398000 according to authority unknown_publisher


True

[ WARN] [1668661767.633273700, 2609.920000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2609.945000 according to authority unknown_publisher
[ WARN] [1668661779.888475473, 2612.446000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2612.475000 according to authority unknown_publisher
[ WARN] [1668661784.190001505, 2613.375000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2613.375000 according to authority unknown_publisher
[ WARN] [1668661784.781982529, 2613.510000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2613.577000 according to authority unknown_publisher
[ WARN] [1668661792.194277314, 2615.196000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2615.244000 according to authority unknown_publisher


In [18]:
joint_goal = arm.get_current_joint_values()
joint_goal[3] = -1.05

arm.set_joint_value_target(joint_goal)
arm.go()

[ WARN] [1668661848.084377892, 2625.200000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2625.200000 according to authority unknown_publisher


True

In [19]:
grip.open()

[ WARN] [1668661878.531465768, 2633.691000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2633.691000 according to authority unknown_publisher
[ WARN] [1668661892.396736567, 2637.657000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2637.657000 according to authority unknown_publisher
[ WARN] [1668661903.858992580, 2640.917000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2640.917000 according to authority unknown_publisher
[ WARN] [1668661912.390161776, 2643.117000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2643.132000 according to authority unknown_publisher


In [ ]:
joint_goal = arm.get_current_joint_values()
joint_goal[3] = 0.0

arm.set_joint_value_target(joint_goal)
arm.go()

In [ ]:
### hacerse hacia atras ####

In [ ]:
joint_goal = arm.get_current_joint_values()
joint_goal[0] = 0.0
joint_goal[1] = 0.0
joint_goal[2] = 0.0
joint_goal[3] = 0.0
joint_goal[4] = 0.0
joint_goal[5] = 0.0

arm.set_joint_value_target(joint_goal)
arm.go()

In [ ]:
#####buscamos mas piedras y repetimos#####